In [1]:
import requests
import pandas as pd
import time
from tqdm import tqdm
import re
import hashlib
import base64

from bs4 import BeautifulSoup
from bs4.dammit import EncodingDetector
from selenium import webdriver

from sklearn.preprocessing import OneHotEncoder
from annoy import AnnoyIndex

In [2]:
import pandas

In [3]:
def get_tables(html_table, tag=['th', 'tr', 'td']):
    table_value = []


    # for my_table in tables:

    # You can find children with multiple tags by passing a list of strings
    rows = html_table.findChildren(tag)

    for row in rows:
        cells = row.findChildren(tag)
        for cell in cells:
            value = cell.string
            if value:
                table_value.append(value.strip())
                # print("The value in this cell is %s" % value)
            else:
                table_value.append("None")
    return dict(zip(table_value[::2], table_value[1::2]))

In [4]:
def parse_page(url):
    
    # get the html file
    page = requests.get(url)
    
    if page.status_code == 200:
        soup = BeautifulSoup(page.content, 'html.parser')

        # get all the tables
        tables = soup.findChildren('table')

        # only the first 4 tables are useful
        tables = tables[:4]


        # UPPER LEFT TABLE
        table_ul = get_tables(tables[0])

        # UPPER RIGHT TABLE
        rows_html = tables[1].findAll("span", {"class": lambda x: x and x.startswith("rating")})
        rows = [x.get_text() for x in rows_html]
        table_ur = dict(zip(rows[::2], rows[1::2]))

#         # BUTTOM TABLES

#         table_b = {}
#         for t in tables[2:]:
#             l = get_tables(t, ['td', 'p', 'tr'])
#             table_b.update(l)


        all_items = {**table_ul, **table_ur}
    
    else:
        print(page.status_code)


    return all_items

In [5]:
def get_date(input_text):
    
    year = re.findall('[0-9]{4}', input_text)
    
    return year

In [6]:
def get_all_boards(gender):
    
#     browser = webdriver.Safari()
    browser = webdriver.Chrome()
    browser.get("https://thegoodride.com/snowboard-reviews/?{}=1".format(gender))
    time.sleep(1)

    button = browser.find_element_by_xpath('/html/body/div[5]/div/div/div/div[3]/form/div[1]/div[42]/a')
    browser.execute_script("arguments[0].click();", button)
    time.sleep(400)
    html = browser.page_source

    browser.close()
    
    return html


In [ ]:
genders = ['mens', 'womens']

for g in genders:
    
    raw_html = get_all_boards(g)
    
    soup = BeautifulSoup(raw_html, "html.parser")
    rows_html = soup.findAll("div", {"class": "board-reviews animate"})

    all_boards = []
        
    for board in rows_html:
        board_name = board.select('h4')[0].text.strip()
        review_url = board.select('a', href=True)[0]['href']
        all_boards.append([board_name, review_url])
        
    df_url = pd.DataFrame(all_boards, columns=['board_name', 'url'])
    print(df_url.shape)

    df_url.to_csv('../data/all_{}_boards.csv'.format(g), index=False)
    
    rating_list = []
    for url in tqdm(df_url['url']):
        d = parse_page(url)
        assert len(d) == 18
        rating_list.append(d)
        time.sleep(.1)
        
    df_rating = pd.DataFrame(rating_list)
    df_final = pd.concat([df_url, df_rating], axis=1)
#     df_final['year'] = df_final['url'].apply(get_date)
    df_final['gender'] = g
    df_final.to_csv('../data/{}_board_20201215.csv'.format(g), index=True)

In [14]:
df_final = pd.read_csv('../data/mens_board_20201215.csv')
df_final['gender'] = 'mens'

In [15]:
df_final_w= pd.read_csv('../data/womens_board_20201215.csv')
df_final_w['gender'] = 'womens'

In [16]:
df_final_w.shape, df_final.shape

((114, 22), (424, 22))

In [17]:
df = pd.concat([df_final, df_final_w], axis=0)
df= df.reset_index(drop=True)

In [18]:
def hashme(x):
    return base64.b64encode(hashlib.sha1(x).digest())

In [19]:
df['id'] = df['url'].astype(str).str.encode('UTF-8').apply(hashme)
df['id'] = df['id'].apply(lambda x: x.decode('utf-8'))

In [20]:
# df = df.drop(columns=['year'])

In [21]:
df.to_csv('../data/all_boards_20201220.csv', index=False)

In [22]:
df = pd.read_csv('../data/all_boards_20201220.csv')

In [23]:
df = df.dropna()
df.columns

Index(['board_name', 'url', 'Overall Rating', 'Riding Style', 'Riding Level',
       'Fits Boot size (US)', 'Manufactured in', 'Shape', 'Camber Profile',
       'Stance', 'Approx. Weight', 'Powder', 'Turning Experience', 'Carving',
       'Speed', 'Uneven Terrain', 'Switch', 'Jumps', 'Jibbing', 'Pipe', 'year',
       'gender', 'id'],
      dtype='object')

In [24]:
df.head()

,board_name,url,Overall Rating,Riding Style,Riding Level,Fits Boot size (US),Manufactured in,Shape,Camber Profile,Stance,...,Carving,Speed,Uneven Terrain,Switch,Jumps,Jibbing,Pipe,year,gender,id
0,Alloy B-Bomb,https://thegoodride.com/snowboard-reviews/allo...,Liked it!,All Mountain Freestyle,Beginner - Expert,8-10,Tunisia by Nidecker,True Twin,Hybrid Camber,Centered,...,Good,Great,Good,Great,Good,Average,Great,"['2016', '2020']",mens,pcQjfBFdQmn6Aqep1PKrAJ8tpZY=
1,Alloy B-Bomb GT,https://thegoodride.com/snowboard-reviews/allo...,None,Freestyle,Intermediate - Expert,8-10,Tunisia by Nidecker,True Twin,Hybrid Camber,Centered,...,Average,Average,Good,Great,Good,Great,Good,['2020'],mens,WdfFXZW/dvMLdhtMZs3ihVDCRHI=
2,Alloy D.O Carbon,https://thegoodride.com/snowboard-reviews/allo...,Pretty Good,Freeride,Expert,8-10,Tunisia by Nidecker,Tapered Directional,Traditional Camber,Setback over 20mm,...,Excellent,Excellent,Poor,Average,Good,Poor,Poor,"['2019', '2020']",mens,GLXwggrHSpW/E/vBriKPps5WPmE=
3,Alloy Darwin Flow,https://thegoodride.com/snowboard-reviews/allo...,None,Alternative Freeride,Advanced - Expert,8-10,Tunisia by Nidecker,Tapered Directional,Directional Camber,Setback over 20mm,...,Great,Great,Great,Poor,Average,Poor,Average,['2020'],mens,vyqJS+sM6x2avNeKZCtTt6aTMlA=
4,Alloy DO 7X,https://thegoodride.com/snowboard-reviews/allo...,Liked it!,Freeride,Expert,8-10,Tunisia by Nidecker,Tapered Directional,Mostly Camber,Setback over 20mm,...,Excellent,Great,Average,Average,Average,Poor,Poor,['2017'],mens,IbVxVGqf7kOWjwmJu3U0lxr16gk=


In [25]:
import boto3
import os
import re
from pathlib import Path
from tqdm import tqdm


from botocore.exceptions import ClientError

def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client('s3')
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client('s3', region_name=region)
            location = {'LocationConstraint': region}
            s3_client.create_bucket(Bucket=bucket_name,
                                    CreateBucketConfiguration=location)
    except ClientError as e:
        logging.error(e)
        return False
    return True

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
#     try:
    response = s3_client.upload_file(file_name, bucket, object_name)
#     exceptions ClientError as e:
#         logging.error(e)
#         return False
    return True

In [26]:
create_bucket("snowboard-finder")

True

In [27]:
upload_file('../data/all_boards_20201220.csv', "snowboard-finder", "all_boards_20201220.csv")

True

# Board Recomendation

In [ ]:
df_final = pd.read_csv('../data/mens_board_20201215.csv')
df_final.shape

In [ ]:
filter_cols = ['Riding Style', 'Riding Level', 'Shape', 'Camber Profile',
       'Stance', 'Approx. Weight', 'Powder', 'Turning Experience', 'Carving',
       'Speed', 'Uneven Terrain', 'Switch', 'Jumps', 'Jibbing', 'Pipe' ]

In [ ]:
df_final.head()

In [ ]:
df_final[filter_cols].head()

In [ ]:
# for col in filter_cols:
#     print(df_final[col].unique())

In [ ]:
# df_final[filter_cols].get_dummies()
df_final_dummies = pd.get_dummies(df_final, columns=filter_cols)
df_final_dummies.shape

In [ ]:
meta_cols = ['board_name', 'url', 'Overall Rating',
             'Fits Boot size (US)', 'Manufactured in', 'year']

### ANNOY

In [ ]:
f = df_final_dummies.shape[1] - len(meta_cols)

In [ ]:
a = AnnoyIndex(f, 'angular')

In [ ]:
for idx, row in df_final_dummies.drop(labels=meta_cols, axis=1).iterrows():
     a.add_item(idx, row.to_list())
#     print(row.values)

In [ ]:
a.build(10)
a.save('../model/annoy_men_angular_20201216.ann')

In [ ]:
u = AnnoyIndex(f, 'angular')
u.load('../model/annoy_men_angular_20201216.ann') # super fast, will just mmap the file

In [ ]:
print(u.get_nns_by_item(173, 1000)) # will find the 1000 nearest neighbors

### Pairwise distance

In [ ]:
from scipy.spatial.distance import squareform, pdist

def distance_function(r):
    dists = pdist(r, 'jaccard')
    
    return pd.DataFrame(squareform(dists))

In [ ]:
df_dist = distance_function(df_final_dummies.drop(labels=meta_cols, axis=1).values)

In [ ]:
reference_index = df_final[df_final['board_name'].str.contains('Jones  Mountain Twin')].index[0]

In [ ]:
target = df_dist.iloc[:, reference_index].nsmallest(6)
target = target.iloc[1:]
target

In [ ]:
df_rec = df_final.iloc[taget_index, :]
df_rec['similarity'] =  target.values

In [ ]:
df_rec